In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('TSLA',
                  start=start_date,
                   end=end_date,
                   progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close",
             "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)



In [ ]:
data.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
3441,2024-05-16,174.100006,175.789993,171.429993,174.839996,174.839996,59812200
3442,2024-05-17,173.550003,179.630005,172.750000,177.460007,177.460007,77445800
3443,2024-05-20,177.559998,177.750000,173.520004,174.949997,174.949997,61727400
3444,2024-05-21,175.509995,186.880005,174.710007,186.600006,186.600006,115266500
3445,2024-05-22,182.850006,183.800003,178.119995,180.110001,180.110001,88085300


In [ ]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],
                                        high=data["High"],
                                        low=data["Low"],
                                        close=data["Close"])])
figure.update_layout(title = "Tesla Stock Price Analysis",
                     xaxis_rangeslider_visible=False)
figure.show()

In [ ]:
correlation=data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    1.000000
Low          0.999686
High         0.999652
Open         0.999223
Date         0.781292
Volume       0.097704
Name: Close, dtype: float64


In [ ]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest, = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 4, 128)            66560     
                                                                 
 lstm_3 (LSTM)               (None, 4, 64)             49408     
                                                                 
 dense_2 (Dense)             (None, 4, 25)             1625      
                                                                 
 dense_3 (Dense)             (None, 4, 1)              26        
                                                                 
Total params: 117619 (459.45 KB)
Trainable params: 117619 (459.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=1, epochs=30)

Epoch 1/30
2756/2756 [==============================] - 28s 9ms/step - loss: 1558.0664
Epoch 2/30
2756/2756 [==============================] - 25s 9ms/step - loss: 123.2423
Epoch 3/30
2756/2756 [==============================] - 23s 8ms/step - loss: 140.6277
Epoch 4/30
2756/2756 [==============================] - 23s 8ms/step - loss: 101.0909
Epoch 5/30
2756/2756 [==============================] - 24s 9ms/step - loss: 110.5764
Epoch 6/30
2756/2756 [==============================] - 23s 8ms/step - loss: 79.9129
Epoch 7/30
2756/2756 [==============================] - 23s 8ms/step - loss: 68.6648
Epoch 8/30
2756/2756 [==============================] - 25s 9ms/step - loss: 68.1004
Epoch 9/30
2756/2756 [==============================] - 25s 9ms/step - loss: 126.7594
Epoch 10/30
2756/2756 [==============================] - 23s 8ms/step - loss: 86.8063
Epoch 11/30
2756/2756 [==============================] - 24s 9ms/step - loss: 72.3618
Epoch 12/30
2756/2756 [==============================] -

In [25]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[177.089996, 180.419998, 177.070007, 74919600]])
model.predict(features)

1/1 [==============================] - 1s 808ms/step


array([[[177.26886],
        [175.11688],
        [185.90945],
        [189.4636 ]]], dtype=float32)